In [2]:
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torchvision.models import Inception_V3_Weights
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import Subset
from src.temperature_scaling import ModelWithTemperature
from src.saps_hyp_opt import lambda_optimization_saps

#  Reprocess: Center Crop and then resize to 299*299
data_transform = transforms.Compose([
    transforms.CenterCrop(299),
    transforms.Resize(299), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)

# load pre-trained model InceptionV3 and set mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1).to(device)
model.eval()

subset_size = len(dataset) // 10
indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_dataset = Subset(dataset, indices)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=2)
model = ModelWithTemperature(model, temperature = 1.0).to(device)
model.set_temperature(train_loader)
model.eval()

Before temperature - NLL: 1.090, ECE: 0.019
Optimal temperature: 0.966
After temperature - NLL: 1.080, ECE: 0.026


ModelWithTemperature(
  (model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x

# $\alpha$=0.1

In [1]:
lambda_values = [0, 1e-3, 0.01, 0.05, 0.1, 0.15, 0.3, 0.5, 1.0]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_saps(model, dataset, lambda_values, device)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")

Before temperature - NLL: 1.064, ECE: 0.023
Optimal temperature: 0.965
After temperature - NLL: 1.055, ECE: 0.029
Looking for optimal lambda...
Optimal lambda is 0.15



# $\alpha$=0.05

In [3]:
lambda_values = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.5]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_saps(model, dataset, lambda_values, device, alpha=0.05)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")

Looking for optimal lambda...
Optimal lambda is 0.2



# $\alpha$=0.2

In [4]:
lambda_values = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.5]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_saps(model, dataset, lambda_values, device, alpha=0.2)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")

Looking for optimal lambda...
Optimal lambda is 0.2

